<img style="float: left;" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Plugins

Decision Intelligence applied in this module:  
* Adding current knowledge (internet) in order to arrive at more informed conclusions  

Semantic Kernel Plugins are complete building blocks of functionality for AI orchestration. Plugins consist of multiple single responsibility functions (semantic or native). A plugin brings together all of the tools it needs a GenAI model, agent or assistant to use under a single construct. Below is an image of a Writer Plugin that consists of multiple functions: Brainstorming, E-mail generation, Translation etc. 

<img width ="600px" src="https://learn.microsoft.com/en-us/semantic-kernel/media/writer-plugin-example.png">


Plugins are very powerful because they allow an AI architect to compose functionality from many smaller functions. Plugins can be crafted in several different ways:
* Use multiple native functions (C# methods) 
* Use multiple semantic functions exposed in prompt files, YAML configurations etc.  
* Hybrid combinations of multiple native or semantic functions  
* Remote services called via functions (database, internet knowledge graph, APIs)  

Learn more about Semantic Kernel Plugins: https://learn.microsoft.com/en-us/semantic-kernel/agents/plugins/ 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets
* Use the API configuration to build the Semantic Kernel orchestrator
* The configuration builder will retrieve the Bing Search API Key

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.13.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.13.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.13.0-alpha"
#r "nuget: Microsoft.Bing.Search.WebSearch, 1.0.0"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var bingSearchAPIKey = config["BingSearch:APIKey"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Bing.Search.WebSearch, 1.0.0 Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.13.0 Microsoft.SemanticKernel.Plugins.Core, 1.13.0-alpha Microsoft.SemanticKernel.Plugins.Web, 1.13.0-alpha

In [2]:
Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Using Azure OpenAI Service


### Step 2 - Limitations of current knowledge of LLMs 

Decisions are being made in our environment and by own selves non-stop. In order to investigate and analyze current decisions, GenAI LLMs need current access to the knowledge graphs that persist this information. 

Execute the cell below to attempt to investigate what coaching decision did the Phoenix Suns make after the 2024 NBA playoffs?"

In [3]:
// Prompt Template for Decision Iquiry
var promptTemplateString = """
What coaching decision did the {{$basketBallTeam}} make after the 2024 NBA playoffs?
Only answer if you know the answer.
""";

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
};

var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["basketBallTeam"] = "Phoenix Suns"
};

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(promptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

As of my last update in October 2023, I don't have information on any coaching decisions made by the Phoenix Suns after the 2024 NBA playoffs. For the most current and accurate information, please refer to the latest news sources or the official Phoenix Suns announcements.

### Step 3 - Acessing Current Knowledge with Bing Plugin

In order to inspect current decisions being made, an LLM needs access to a knowledge graph (internet) that includes recent information and news. Conveniently Semantic Kernel includes a web plugin that exposes access to the Bing Search knowledge graph.

Execute the two cells below to create and import the Bing connector to Semantic Kernel instance. **Notice that just by adding the Bing plugin, the link between Semantic Kernel knowledge and the LLM is still not made.**    

In [4]:
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

#pragma warning disable SKEXP0050
var bingConnector = new BingConnector(bingSearchAPIKey);
var bingSearchPlugin = new WebSearchEnginePlugin(bingConnector);

semanticKernel.ImportPluginFromObject(bingSearchPlugin, "bing");
#pragma warning restore SKEXP0050

In [5]:
// Still doesn't work, even though Bing Knowledge Graph is enabled
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(promptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

As of my last update in October 2023, I do not have information on any coaching decisions made by the Phoenix Suns after the 2024 NBA playoffs. For the most current and accurate information, please refer to the latest news sources or the official Phoenix Suns announcements.

### Step 4 - Proving Semantic Kernel the ability to call the Bing Search Plugin

Notice after importing the Bing Search plugin into the Semantic Kernel instance, the LLM still was not able to answer the question. What is missing? **In addition to importing the appropriate plugin, the LLM needs to be "allowed" to automatically invoke imported Semantic Kernel functions. This is done by overriding the ToolCallBehavior property to AutoInvokeFunctions.**  

Execute the cell below to allow auto invocation of Semantic Kernel functions. The configuration now allows Semantic Kernel to find the right tool (Bing Search) and automatically invoke it.

In [6]:
// Prompt Template for Decision Iquiry
var promptTemplateString = """
What coaching decision did {{$basketBallTeam}} make after the 2024 playoffs? 
Only answer if you know the answer.
""";

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    // Allows it to call the available Bing Plugin
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["basketBallTeam"] = "Phoenix Suns"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(promptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

After the 2024 playoffs, the Phoenix Suns fired head coach Frank Vogel following a disappointing season that ended with a first-round sweep by the Minnesota Timberwolves. Subsequently, the Suns hired Mike Budenholzer as their new head coach.

### Step 5 - Understanding & Explaining the Reasoning
> "The key to good decision making is not knowledge. It is understanding. We are swimming in the former. We are desperately lacking in the latter."
>
> -- <cite>Malcolm Gladwell (journalist, author, and public speaker known for his thought-provoking books)</cite> 

In the previous step, Semantic Kernel with the Bing plugin and Auto Functions turned on where able to provide a seemingly "updated" answer. However, this answer can be improved further. In this next step, we will extract the understanging component from the GenAI LLM. Understanding refers to the ability to interpret, connect, and make sense of that knowledge. It's the ability to see the relationships between pieces of information, to recognize patterns, and to apply knowledge in context.

How can understanding be extracted from GenAI? **Instructing the GenAI system to communicate the decision or recommendation with a framework like the Minto Pyramid can provide an clear way illustrate the reasoning process to the user.** More information about the Minto Pyramid communication framework can be found on the Untools.co website: https://untools.co/minto-pyramid/    

<img width ="600px" src="https://assets-us-01.kc-usercontent.com/c6e42f10-0ed4-0062-585c-b740aa1ad46c/b3bbf2af-f6b2-4679-8a3a-837e37250990/minto-pyramid.png">

Execute the cell below to apply the Minto Pyramid Principle as guide for the GenAI model to structure the answer.  

In [7]:
// Prompt Template for Decision Iquiry
// Note the additional instruction to use the Minto Pyramid Principle
var promptTemplateString = """
What coaching decision did {{$basketBallTeam}} make after the 2024 playoffs? 
Only answer if you know the answer.
Use the Minto Pyramid Principle to structure your answer.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["basketBallTeam"] = "Phoenix Suns"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(promptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

**Key Point:**
The Phoenix Suns made a significant coaching decision after the 2024 playoffs by firing head coach Frank Vogel.

**Supporting Points:**
1. **Reason for Decision:**
   - The decision came after a disappointing season that ended without a playoff victory.
   - The Suns were swept in the first round of the playoffs by the Minnesota Timberwolves.

2. **Outcome:**
   - Frank Vogel's tenure as head coach lasted only one season.
   - The team decided to part ways with Vogel less than two weeks after their playoff exit.

3. **Future Steps:**
   - The Suns have a head-coaching vacancy and will be looking to fill the position to improve their performance in the upcoming seasons.

**Conclusion:**
The Phoenix Suns' decision to fire Frank Vogel was driven by the team's underperformance and early playoff exit, highlighting the franchise's intent to seek new leadership to achieve better results.

### Step 6 - Understanding Auto Function Calling Behavior

How does Semantic Kernel and the LLM know to call the function? What if there are multiple functions or plugins, how does LLM signal Semantic Kernel which function to call? 

Executing the cell below will illustrate that the invocation of the functions is not magic, but guided by the descriptions of the plugin function descriptions and parameter descriptions. Note that the Search function is described as "Perform a Web Search". This provides a big clue to Semantic Kernel that this a possible tool (function) to use to extract current information.

In [8]:
// Method for discovering the available functions loaded into Semantic Kernel via metadata
private void PrintFunction(KernelFunctionMetadata func)
{
    Console.WriteLine($"Plugin: {func.PluginName}");
    Console.WriteLine($"  Function Name: {func.Name}");
    Console.WriteLine($"  Function Description: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("    Function Parameters:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"      - {p.Name}: {p.Description}");
            Console.WriteLine($"        default: '{p.DefaultValue}'");
        }
    }

    Console.WriteLine();
}
// Get the functions metadata
var functions = semanticKernel.Plugins.GetFunctionsMetadata();

foreach (KernelFunctionMetadata func in functions)
{
    PrintFunction(func);
}

Plugin: bing
  Function Name: Search
  Function Description: Perform a web search.
    Function Parameters:
      - query: Search query
        default: ''
      - count: Number of results
        default: '10'
      - offset: Number of results to skip
        default: '0'

Plugin: bing
  Function Name: GetSearchResults
  Function Description: Perform a web search and return complete results.
    Function Parameters:
      - query: Text to search for
        default: ''
      - count: Number of results
        default: '1'
      - offset: Number of results to skip
        default: '0'

